In [1]:
import cobra

/Users/helenscott/opt/miniconda3/envs/med4-hot1a3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
model = cobra.io.read_sbml_model("model.xml")

'' is not a valid SBML 'SId'.


In [3]:
# This is the list of reactions that are involved in cycles from MEMOTE
cycle_rxns = ["rxn00836_c0","rxn00423_c0","rxn03239_c0","rxn01867_c0","rxn00545_c0","rxn05040_c0","rxn01297_c0","rxn08941_c0","rxn05733_c0","rxn02270_c0","rxn00649_c0","rxn01739_c0","rxn07434_c0","rxn05316_c0","rxn00225_c0","rxn03436_c0","rxn00781_c0","rxn00097_c0","rxn02212_c0","rxn00692_c0","rxn02720_c0","rxn03958_c0","rxn03174_c0","rxn01354_c0","rxn08799_c0","rxn03419_c0","rxn00272_c0","rxn06078_c0","rxn05595_c0","rxn03245_c0","rxn00907_c0","rxn02911_c0","rxn09188_c0","rxn00248_c0","rxn00392_c0","rxn00747_c0","rxn06777_c0","rxn03240_c0","rxn00555_c0","rxn03435_c0","rxn03080_c0","rxn00849_c0","rxn00737_c0","rxn10162_c0","rxn00777_c0","rxn01740_c0","rxn00251_c0","rxn00237_c0","rxn03175_c0","rxn00299_c0","rxn00904_c0","rxn00786_c0","rxn01100_c0","rxn00102_c0","rxn01255_c0","rxn01211_c0","rxn03253_c0","rxn02011_c0","rxn00173_c0","rxn09176_c0","rxn01669_c0","rxn03135_c0","rxn00915_c0","rxn01116_c0","rxn07435_c0","rxn01678_c0","rxn02286_c0","rxn00832_c0","rxn01106_c0","rxn00650_c0","rxn03437_c0","rxn01353_c0","rxn00616_c0","rxn02934_c0","rxn00420_c0","rxn02804_c0","rxn00048_c0","rxn00283_c0","rxn00184_c0","rxn00278_c0","rxn01333_c0","rxn00407_c0","rxn03243_c0","rxn02285_c0","rxn00459_c0","rxn00076_c0","rxn05579_c0","rxn02803_c0","rxn00461_c0","rxn00350_c0","rxn03137_c0","rxn09234_c0","rxn02476_c0","rxn04954_c0","rxn00903_c0","rxn02213_c0","rxn10202_c0","rxn00704_c0","rxn01332_c0","rxn00785_c0","rxn00693_c0","rxn03909_c0","rxn01101_c0","rxn00558_c0","rxn00077_c0","rxn03638_c0","rxn01068_c0","rxn03242_c0","rxn00851_c0","rxn01200_c0","rxn01241_c0","rxn02473_c0","rxn00122_c0","rxn03241_c0","rxn00085_c0","rxn02008_c0","rxn01225_c0","rxn00946_c0","rxn05114_c0","rxn03164_c0","rxn00929_c0","rxn08942_c0","rxn10770_c0","rxn00504_c0","rxn01485_c0","rxn05206_c0","rxn00239_c0","rxn00302_c0","rxn00623_c0","rxn08043_c0","rxn08131_c0","rxn01137_c0","rxn06510_c0","rxn00274_c0","rxn06335_c0","rxn02914_c0","rxn09232_c0","rxn06075_c0","rxn05289_c0","rxn00117_c0","rxn00330_c0","rxn00293_c0","rxn00154_c0","rxn08213_c0","rxn08173_c0","rxn00064_c0","rxn09260_c0","rxn09330_c0","EX_cpd00268_e0","EX_cpd00098_e0","EX_cpd00246_e0","EX_cpd00214_e0"]

In [7]:
# Find all reactions that involve PRPP (what rxn00836 produces)
metabolite_id = "cpd00103_c0"
# Check if the metabolite exists in the model
if metabolite_id in model.metabolites:
    metabolite = model.metabolites.get_by_id(metabolite_id)
else:
    raise ValueError(f"Metabolite {metabolite_id} not found in the model.")

# Find all reactions containing the metabolite
reactions_with_metabolite = [reaction.id for reaction in model.reactions if metabolite in reaction.metabolites if reaction.id in cycle_rxns]


In [8]:
reactions_with_metabolite

['rxn00836_c0', 'rxn00915_c0']

In [9]:
model.reactions.rxn00836_c0

Reaction identifier,rxn00836_c0
Name,IMP:diphosphate phospho-D-ribosyltransferase [c0]
Memory address,0x17da37b80
Stoichiometry,cpd00012_c0 + cpd00067_c0 + cpd00114_c0 <-- cpd00103_c0 + cpd00226_c0 PPi [c0] + H+ [c0] + IMP [c0] <-- PRPP [c0] + HYXN [c0]
GPR,MIT1002_01649
Lower bound,-1000.0
Upper bound,0.0


In [11]:
model.reactions.rxn00915_c0

Reaction identifier,rxn00915_c0
Name,GMP:diphosphate 5-phospho-alpha-D-ribosyltransferase [c0]
Memory address,0x17db3d2d0
Stoichiometry,cpd00012_c0 + cpd00067_c0 + cpd00126_c0 <-- cpd00103_c0 + cpd00207_c0 PPi [c0] + H+ [c0] + GMP [c0] <-- PRPP [c0] + Guanine [c0]
GPR,MIT1002_01649
Lower bound,-1000.0
Upper bound,0.0


In [12]:
# Find all reactions that involve IMP (what rxn00836 uses)
metabolite_id = "cpd00114_c0"
# Check if the metabolite exists in the model
if metabolite_id in model.metabolites:
    metabolite = model.metabolites.get_by_id(metabolite_id)
else:
    raise ValueError(f"Metabolite {metabolite_id} not found in the model.")

# Find all reactions containing the metabolite
reactions_with_metabolite = [reaction.id for reaction in model.reactions if metabolite in reaction.metabolites if reaction.id in cycle_rxns]
reactions_with_metabolite

['rxn00836_c0', 'rxn00832_c0']

In [13]:
model.reactions.rxn00832_c0

Reaction identifier,rxn00832_c0
Name,"IMP 1,2-hydrolase (decyclizing) [c0]"
Memory address,0x17deb6470
Stoichiometry,cpd00001_c0 + cpd00114_c0 <=> cpd02884_c0 H2O [c0] + IMP [c0] <=> FAICAR [c0]
GPR,MIT1002_03959
Lower bound,-1000.0
Upper bound,1000.0


In [14]:
# Find all reactions that involve IMP (what rxn00836 uses)
metabolite_id = "cpd02884_c0"
# Check if the metabolite exists in the model
if metabolite_id in model.metabolites:
    metabolite = model.metabolites.get_by_id(metabolite_id)
else:
    raise ValueError(f"Metabolite {metabolite_id} not found in the model.")

# Find all reactions containing the metabolite
reactions_with_metabolite = [reaction.id for reaction in model.reactions if metabolite in reaction.metabolites if reaction.id in cycle_rxns]
reactions_with_metabolite

['rxn00832_c0', 'rxn03137_c0']

In [15]:
model.reactions.rxn03137_c0

Reaction identifier,rxn03137_c0
Name,10-Formyltetrahydrofolate:5'-phosphoribosyl-5-amino-4-imidazolecarboxamide formyltransferase [c0]
Memory address,0x17e1994e0
Stoichiometry,cpd00201_c0 + cpd02851_c0 <=> cpd00087_c0 + cpd02884_c0 10-Formyltetrahydrofolate [c0] + AICAR [c0] <=> Tetrahydrofolate [c0] + FAICAR [c0]
GPR,MIT1002_03959
Lower bound,-1000.0
Upper bound,1000.0


In [17]:
# Find all reactions that involve IMP (what rxn00836 uses)
metabolite_id = "cpd02851_c0"
# Check if the metabolite exists in the model
if metabolite_id in model.metabolites:
    metabolite = model.metabolites.get_by_id(metabolite_id)
else:
    raise ValueError(f"Metabolite {metabolite_id} not found in the model.")

# Find all reactions containing the metabolite
reactions_with_metabolite = [reaction.id for reaction in model.reactions if metabolite in reaction.metabolites if reaction.id in cycle_rxns]
reactions_with_metabolite

['rxn03135_c0', 'rxn03137_c0']

In [18]:
model.reactions.rxn03135_c0

Reaction identifier,rxn03135_c0
Name,R04558 [c0]
Memory address,0x17daf3130
Stoichiometry,cpd00023_c0 + 2.0 cpd00067_c0 + cpd02843_c0 + cpd02851_c0 <-- cpd00053_c0 + cpd02991_c0 L-Glutamate [c0] + 2.0 H+ [c0] + D-erythro-imidazol-glycerol-phosphate [c0] + AICAR [c0] <-- L-Glutamine [c0] + phosphoribulosylformimino-AICAR-phosphate [c0]
GPR,MIT1002_02636 and MIT1002_02638 and MIT1002_01061
Lower bound,-1000.0
Upper bound,0.0


In [22]:
s_matrix = cobra.util.create_stoichiometric_matrix(model)
s_matrix

array([[-1.,  0.,  0., ...,  0.,  0.,  0.],
       [-1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., -1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [24]:
# With these reactions, am I functionally dealing with a directed or undirected graph?
# Are the bounds for the reactions all set as reversible?

for reaction in cycle_rxns:
    print(reaction + " | " + str(model.reactions.get_by_id(reaction).bounds))

rxn00836_c0 | (-1000.0, 0.0)
rxn00423_c0 | (0.0, 1000.0)
rxn03239_c0 | (-1000.0, 1000.0)
rxn01867_c0 | (0.0, 1000.0)
rxn00545_c0 | (0.0, 1000.0)
rxn05040_c0 | (0.0, 1000.0)
rxn01297_c0 | (-1000.0, 1000.0)
rxn08941_c0 | (-1000.0, 1000.0)
rxn05733_c0 | (0.0, 1000.0)
rxn02270_c0 | (-1000.0, 0.0)
rxn00649_c0 | (0.0, 1000.0)
rxn01739_c0 | (0.0, 1000.0)
rxn07434_c0 | (0.0, 1000.0)
rxn05316_c0 | (-1000.0, 1000.0)
rxn00225_c0 | (-1000.0, 1000.0)
rxn03436_c0 | (-1000.0, 1000.0)
rxn00781_c0 | (-1000.0, 1000.0)
rxn00097_c0 | (-1000.0, 1000.0)
rxn02212_c0 | (0.0, 1000.0)
rxn00692_c0 | (-1000.0, 1000.0)
rxn02720_c0 | (-1000.0, 1000.0)
rxn03958_c0 | (-1000.0, 1000.0)
rxn03174_c0 | (-1000.0, 0.0)
rxn01354_c0 | (-1000.0, 1000.0)
rxn08799_c0 | (-1000.0, 1000.0)
rxn03419_c0 | (-1000.0, 1000.0)
rxn00272_c0 | (-1000.0, 1000.0)
rxn06078_c0 | (0.0, 1000.0)
rxn05595_c0 | (-1000.0, 1000.0)
rxn03245_c0 | (-1000.0, 1000.0)
rxn00907_c0 | (-1000.0, 1000.0)
rxn02911_c0 | (-1000.0, 1000.0)
rxn09188_c0 | (-1000.0, 1

In [25]:
n_rev = 0
n_irrev = 0

for reaction in cycle_rxns:
    if model.reactions.get_by_id(reaction).reversibility:
        n_rev += 1
    else:
        n_irrev += 1

print("Reversible: " + str(n_rev))
print("Irreversible: " + str(n_irrev))

Reversible: 99
Irreversible: 53


In [29]:
# Define function that gives all of the reactions one metabolite away from the
# input reaction
def find_neighbor_rxns(rxn, cycle_rxns):
    neighbor_rxns = []
    for met in rxn.metabolites:
        for neighbor_rxn in met.reactions:
            if neighbor_rxn.id not in neighbor_rxns and neighbor_rxn.id in cycle_rxns:
                neighbor_rxns.append(neighbor_rxn)
    return neighbor_rxns

In [31]:
find_neighbor_rxns(model.reactions.rxn00836_c0, cycle_rxns)

[<Reaction rxn00915_c0 at 0x17db3d2d0>,
 <Reaction rxn00293_c0 at 0x17e4bf700>,
 <Reaction rxn00122_c0 at 0x17e2e7850>,
 <Reaction rxn09176_c0 at 0x17daaba60>,
 <Reaction rxn00836_c0 at 0x17da37b80>,
 <Reaction rxn05733_c0 at 0x17db88ac0>,
 <Reaction rxn00504_c0 at 0x17e3f0dc0>,
 <Reaction rxn00781_c0 at 0x17dbf8fa0>,
 <Reaction rxn02285_c0 at 0x17e119030>,
 <Reaction rxn00392_c0 at 0x17dd41090>,
 <Reaction rxn03958_c0 at 0x17dc492d0>,
 <Reaction rxn01867_c0 at 0x17daa9a80>,
 <Reaction rxn06777_c0 at 0x17dd41c00>,
 <Reaction rxn02270_c0 at 0x17db89d80>,
 <Reaction rxn00239_c0 at 0x17e3f1ea0>,
 <Reaction rxn01211_c0 at 0x17de89ff0>,
 <Reaction rxn00299_c0 at 0x17de03010>,
 <Reaction rxn03135_c0 at 0x17daf3130>,
 <Reaction rxn05040_c0 at 0x17daf37c0>,
 <Reaction rxn00623_c0 at 0x17e3f37f0>,
 <Reaction rxn00248_c0 at 0x17dcfb850>,
 <Reaction rxn01354_c0 at 0x17dc4ba00>,
 <Reaction rxn02720_c0 at 0x17dc23b80>,
 <Reaction rxn00097_c0 at 0x17dbfbd60>,
 <Reaction rxn08799_c0 at 0x17dcbc370>,


In [32]:
model.reactions.rxn00293_c0

Reaction identifier,rxn00293_c0
Name,UTP:N-acetyl-alpha-D-glucosamine-1-phosphate uridylyltransferase [c0]
Memory address,0x17e4bf700
Stoichiometry,cpd00062_c0 + cpd02611_c0 --> cpd00012_c0 + cpd00037_c0 UTP [c0] + N-Acetyl-D-glucosamine1-phosphate [c0] --> PPi [c0] + UDP-N-acetylglucosamine [c0]
GPR,MIT1002_03899
Lower bound,0.0
Upper bound,1000.0


In [37]:
nodes_visited = []
new_nodes = [model.reactions.rxn00836_c0]
for node in new_nodes:
    if node in nodes_visited:
        print("Cycle found")
        print(nodes_visited)
        break
    nodes_visited = nodes_visited + new_nodes
    new_nodes = find_neighbor_rxns(node, cycle_rxns)

In [38]:
def find_cycles(start_rxn, cycle_rxns, visited=None, path=None):
    if visited is None:
        visited = set()
    if path is None:
        path = []

    path.append(start_rxn.id)
    visited.add(start_rxn.id)

    neighbors = find_neighbor_rxns(start_rxn, cycle_rxns)
    for neighbor in neighbors:
        if neighbor.id not in visited:
            cycle = find_cycles(neighbor, cycle_rxns, visited, path)
            if cycle:
                return cycle
        elif neighbor.id in path:
            return path[path.index(neighbor.id):]

    path.pop()
    return None

In [39]:
start_reaction = model.reactions.get_by_id("rxn00836_c0")  # Replace with your starting reaction
cycle = find_cycles(start_reaction, cycle_rxns)

In [40]:
cycle

['rxn00915_c0']